# Poisson equation with pure Neumann boundary conditions

This demo is implemented in a single Python file,
demo\_neumann-poisson.py, which contains both the variational form and
the solver.

## Implementation

This description goes through the implementation in
demo\_neumann-poisson.py of a solver for the above described Poisson
equation step-by-step.

First, the :pydolfin module is imported:

In [ ]:
from dolfin import *

We proceed by defining a mesh of the domain. We use a built-in mesh
provided by the class :pyUnitSquareMesh
&lt;dolfin.cpp.UnitSquareMesh&gt;. In order to create a mesh consisting
of $64 \times 64$ squares with each square divided into two triangles,
we do as follows:

In [ ]:
# Create mesh
mesh = UnitSquareMesh(64, 64)

Next, we need to define the function spaces. We define the two function
spaces $V$ and $R$ separately, before combining these into a mixed
function space $W$:

In [ ]:
# Define function spaces and mixed (product) space
V = FunctionSpace(mesh, "CG", 1)
R = FunctionSpace(mesh, "R", 0)
W = V * R

The second argument to :pyFunctionSpace
&lt;dolfin.functions.functionspace.FunctionSpace&gt; specifies the type
of finite element family, while the third argument specifies the
polynomial degree. The UFL user manual contains a list of all available
finite element families and more details. The \* operator creates a
mixed (product) space $W$ from the two separate spaces $V$ and $R$.
Hence,

$$W = \{ (v, d) \ \text{such that} \ v \in V, d \in R \}.$$

Now, we want to define the variational problem, but first we need to
specify the trial functions (the unknowns) and the test functions. This
can be done using
:pyTrialFunctions&lt;dolfin.functions.function.TrialFunction&gt; and
:pyTestFunctions
&lt;dolfin.functions.function.TestFunction&gt;. It only remains to
define the source function $f$, before we define the bilinear and linear
forms. It is given by a simple mathematical formula, and can easily be
declared using the :pyExpression
&lt;dolfin.functions.expression.Expression&gt; class. Note that the
string defining `f` uses C++ syntax since, for efficiency, DOLFIN will
generate and compile C++ code for these expressions at run-time. The
following code shows how this is done and defines the variational
problem:

In [ ]:
# Define variational problem
(u, c) = TrialFunction(W)
(v, d) = TestFunctions(W)
f = Expression("10*exp(-(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2)) / 0.02)")
g = Expression("-sin(5*x[0])")
a = (inner(grad(u), grad(v)) + c*v + u*d)*dx
L = f*v*dx + g*v*ds

Since we have natural (Neumann) boundary conditions in this problem, we
don´t have to implement boundary conditions. This is because Neumann
boundary conditions are default in DOLFIN.

To compute the solution we use the bilinear form, the linear forms, and
the boundary condition, but we also need to create a
:pyFunction &lt;dolfin.functions.function.Function&gt; to store the
solution(s). The (full) solution will be stored in `w`, which we
initialize using the
:pyFunctionSpace&lt;dolfin.functions.functionspace.FunctionSpace&gt;
`W`. The actual computation is performed by calling
:pysolve&lt;dolfin.fem.solving.solve&gt;. The separate components `u`
and `c` of the solution can be extracted by calling the split function.
Finally, we plot the solutions to examine the result.

In [ ]:
# Compute solution
w = Function(W)
solve(a == L, w)
(u, c) = w.split()

# Plot solution
plot(u, interactive=True)

## Complete code